In [1]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).

GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)

20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
'''

# from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from scribe_classifier.data.scribe.readers.dbhandler import DataFramePickler
from scribe_classifier.data.canada import TitleSet, TitleRecord
from scribe_classifier.data.canada import AllCodes, CodeRecord
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.optimizers import RMSprop
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, cross_validate
from floatrange import floatrange
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [3]:
BASE_DIR = '.'
GLOVE_DIR = BASE_DIR + '/source_data/wiki_glove/'
TEXT_DATA_DIR = BASE_DIR + '/source_data/20_newsgroup/'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
target_level = 3

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

all_codes = AllCodes.load_from_pickle('source_data/pickles/canada/tidy_sets/all_codes.P', is_path=True)
ac_vec = all_codes.get_codes_for_level(target_level=target_level)
ac_vec.append('NA')
tset = TitleSet.load_from_pickle('source_data/pickles/canada/tidy_sets/all_titles.P',is_path=True)
tset = tset.copy_and_append_empty_string_class()

Indexing word vectors.
Found 400000 word vectors.
Processing text dataset


In [12]:
texts = []  # list of text samples
labels = []  # list of label ids
# labels_index = {}  # dictionary mapping label name to numeric id

# lbl_enc = LabelEncoder()
# oh_enc = OneHotEncoder()
lbl_bin = LabelBinarizer()
texts = tset.get_title_vec()
text_labels = tset.get_code_vec(target_level=target_level)
lbl_bin.fit(ac_vec)
# labels = lbl_bin.transform(text_labels)
labels = np.asarray(text_labels)
# labels = np.ndarray(text_labels)
# labels = lbl_enc.fit_transform(np.asarray(text_labels)).reshape(-1, 1)
# labels.shape

In [13]:
# for i in range(len(ac_vec)):
#     labels_index[i] = ac_vec[i]

print('Found %s texts.' % len(texts))

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# ohenc_labels = oh_enc.fit_transform(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Found 48666 texts.
Found 7945 unique tokens.
Shape of data tensor: (48666, 1000)
Shape of label tensor: (48666,)


In [56]:
print(data[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [51]:
# split the data into a training set and a validation set
split_out = train_test_split(data, labels, texts, train_size=0.80, test_size=0.20, shuffle=True, stratify=text_labels)
x_val = split_out[1]
x_train = split_out[0]
y_val = split_out[3]
y_train = split_out[2]
val_texts = split_out[5]
train_texts = split_out[4]
sparse.csr_matrix(x_train)
sparse.csr_matrix(x_val)
print("train_X: ", x_train.shape)
print("train_Y: ", y_train.shape)
print("train_texts", len(train_texts))
print("val_X", x_val.shape)
print("val_Y", y_val.shape)
print("val_texts", len(val_texts))

train_X:  (38932, 1000)
train_Y:  (38932,)
train_texts 38932
val_X (9734, 1000)
val_Y (9734,)
val_texts 9734


In [52]:
vect = CountVectorizer(stop_words='english', ngram_range=(1, 3))
vect_train_x = vect.fit_transform(train_texts)
vect_val_x = vect.transform(val_texts)
print(vect_train_x.shape)
print(vect_val_x.shape)

(38932, 73650)
(9734, 73650)


In [53]:
X_train = sparse.hstack((vect_train_x, x_train))
X_val = sparse.hstack((vect_val_x,x_val)).toarray()
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
# print(y_train)

(38932, 74650)
(9734, 74650)
(38932,)


In [44]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)
all_x = sparse.vstack((X_train, X_val))
all_y = np.append(y_train, y_val)
print(all_x.shape)
print(all_y.shape)

(38933, 74503)
(9733, 74503)
(38933,)
(9733,)
(48666, 74503)
(48666,)


In [54]:
from sklearn.linear_model import RidgeClassifierCV
cv = StratifiedShuffleSplit(train_size=0.10, test_size=0.90)
clf = SGDClassifier(alpha=1e-4, max_iter=1000, tol=1e-4, verbose=3)

In [55]:
from sklearn.model_selection import cross_val_score

cross_val_score(clf, vect_train_x, y=y_train, scoring='accuracy', cv=cv, verbose=0)

-- Epoch 1
Norm: 36.40, NNZs: 331, Bias: -1.670061, T: 3893, Avg. loss: 0.052958
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 22.94, NNZs: 397, Bias: -1.728009, T: 7786, Avg. loss: 0.007447
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 17.69, NNZs: 451, Bias: -1.752865, T: 11679, Avg. loss: 0.001761
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 14.38, NNZs: 464, Bias: -1.727110, T: 15572, Avg. loss: 0.000476
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 12.16, NNZs: 476, Bias: -1.717615, T: 19465, Avg. loss: 0.000678
Total training time: 0.01 seconds.
Convergence after 5 epochs took 0.01 seconds
-- Epoch 1
Norm: 38.02, NNZs: 342, Bias: -1.567295, T: 3893, Avg. loss: 0.050756
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 25.68, NNZs: 457, Bias: -1.748482, T: 7786, Avg. loss: 0.008909
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 18.57, NNZs: 472, Bias: -1.768009, T: 11679, Avg. loss: 0.001149
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 14.81,

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Norm: 33.02, NNZs: 270, Bias: -1.393416, T: 3893, Avg. loss: 0.027967
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 21.30, NNZs: 336, Bias: -1.421973, T: 7786, Avg. loss: 0.006547
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 15.91, NNZs: 370, Bias: -1.411073, T: 11679, Avg. loss: 0.002421
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 12.89, NNZs: 387, Bias: -1.397168, T: 15572, Avg. loss: 0.001404
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 11.77, NNZs: 449, Bias: -1.413581, T: 19465, Avg. loss: 0.001789
Total training time: 0.01 seconds.
Convergence after 5 epochs took 0.01 seconds
-- Epoch 1
Norm: 29.83, NNZs: 215, Bias: -1.177774, T: 3893, Avg. loss: 0.024503
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 18.50, NNZs: 264, Bias: -1.311966, T: 7786, Avg. loss: 0.004867
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 13.48, NNZs: 274, Bias: -1.292031, T: 11679, Avg. loss: 0.000450
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 10.96, NNZs: 279,

Norm: 36.62, NNZs: 333, Bias: -1.538787, T: 3893, Avg. loss: 0.041503
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 23.41, NNZs: 417, Bias: -1.668819, T: 7786, Avg. loss: 0.007909
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 17.60, NNZs: 438, Bias: -1.650421, T: 11679, Avg. loss: 0.001671
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 14.08, NNZs: 456, Bias: -1.666816, T: 15572, Avg. loss: 0.000757
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 12.16, NNZs: 459, Bias: -1.639056, T: 19465, Avg. loss: 0.000786
Total training time: 0.01 seconds.
Convergence after 5 epochs took 0.01 seconds
-- Epoch 1
Norm: 32.83, NNZs: 264, Bias: -1.328670, T: 3893, Avg. loss: 0.031033
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 21.69, NNZs: 353, Bias: -1.442720, T: 7786, Avg. loss: 0.008836
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 16.30, NNZs: 379, Bias: -1.433559, T: 11679, Avg. loss: 0.002336
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 13.36, NNZs: 414,

Norm: 48.93, NNZs: 568, Bias: -1.488613, T: 3893, Avg. loss: 0.090188
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 32.48, NNZs: 730, Bias: -1.650736, T: 7786, Avg. loss: 0.015553
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 23.83, NNZs: 771, Bias: -1.658630, T: 11679, Avg. loss: 0.002557
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 20.28, NNZs: 838, Bias: -1.615425, T: 15572, Avg. loss: 0.003280
Total training time: 0.01 seconds.
Convergence after 4 epochs took 0.01 seconds
-- Epoch 1
Norm: 37.19, NNZs: 353, Bias: -1.303447, T: 3893, Avg. loss: 0.047737
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 23.82, NNZs: 443, Bias: -1.410807, T: 7786, Avg. loss: 0.006207
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 17.33, NNZs: 460, Bias: -1.390303, T: 11679, Avg. loss: 0.002144
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 13.61, NNZs: 469, Bias: -1.390477, T: 15572, Avg. loss: 0.000325
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 13.10, NNZs: 522,

...

Norm: 7.98, NNZs: 200, Bias: -1.349206, T: 19465, Avg. loss: 0.000325
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 6.71, NNZs: 200, Bias: -1.349206, T: 23358, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 5.92, NNZs: 200, Bias: -1.345174, T: 27251, Avg. loss: 0.000086
Total training time: 0.01 seconds.
Convergence after 7 epochs took 0.01 seconds
-- Epoch 1
Norm: 32.13, NNZs: 257, Bias: -1.387786, T: 3893, Avg. loss: 0.032343
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 20.49, NNZs: 316, Bias: -1.499970, T: 7786, Avg. loss: 0.004986
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 15.11, NNZs: 335, Bias: -1.499496, T: 11679, Avg. loss: 0.001585
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 12.97, NNZs: 372, Bias: -1.511673, T: 15572, Avg. loss: 0.000570
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 11.88, NNZs: 396, Bias: -1.501133, T: 19465, Avg. loss: 0.000688
Total training time: 0.01 seconds.
Convergence after 5 epochs took 0.0

-- Epoch 4
Norm: 11.56, NNZs: 276, Bias: -1.377021, T: 15572, Avg. loss: 0.001101
Total training time: 0.01 seconds.
Convergence after 4 epochs took 0.01 seconds
-- Epoch 1
Norm: 61.77, NNZs: 881, Bias: -2.254091, T: 3893, Avg. loss: 0.151918
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 40.69, NNZs: 1108, Bias: -2.461766, T: 7786, Avg. loss: 0.022171
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 31.19, NNZs: 1213, Bias: -2.468767, T: 11679, Avg. loss: 0.006871
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 25.24, NNZs: 1251, Bias: -2.443565, T: 15572, Avg. loss: 0.002253
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 21.27, NNZs: 1268, Bias: -2.426300, T: 19465, Avg. loss: 0.002118
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 18.75, NNZs: 1289, Bias: -2.399893, T: 23358, Avg. loss: 0.001251
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 16.87, NNZs: 1318, Bias: -2.388698, T: 27251, Avg. loss: 0.001049
Total training time: 0.01 seconds.
-- Epoch 8
Norm

Norm: 5.82, NNZs: 89, Bias: -1.012902, T: 15572, Avg. loss: 0.000000
Total training time: 0.01 seconds.
Convergence after 4 epochs took 0.01 seconds
-- Epoch 1
Norm: 30.66, NNZs: 233, Bias: -1.421780, T: 3893, Avg. loss: 0.047550
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 21.69, NNZs: 327, Bias: -1.540353, T: 7786, Avg. loss: 0.007877
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 16.79, NNZs: 349, Bias: -1.542411, T: 11679, Avg. loss: 0.002948
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 13.33, NNZs: 366, Bias: -1.550320, T: 15572, Avg. loss: 0.000589
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 11.22, NNZs: 369, Bias: -1.540590, T: 19465, Avg. loss: 0.000522
Total training time: 0.01 seconds.
Convergence after 5 epochs took 0.01 seconds
-- Epoch 1
Norm: 22.21, NNZs: 121, Bias: -1.081733, T: 3893, Avg. loss: 0.011798
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 13.56, NNZs: 142, Bias: -1.120679, T: 7786, Avg. loss: 0.001991
Total training time: 0.00 

[Parallel(n_jobs=1)]: Done 141 out of 141 | elapsed:    1.2s finished


array([ 0.19997717,  0.19997717,  0.20128999,  0.20000571,  0.19997717,
        0.19997717,  0.19997717,  0.19997717,  0.19997717,  0.19997717])

In [47]:
clf.fit(vect_train_x, y_train)
train_preds = clf.predict(vect_train_x)
val_preds = clf.predict(vect_val_x)

-- Epoch 1
Norm: 11.45, NNZs: 1381, Bias: -1.677538, T: 38933, Avg. loss: 0.013829
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 8.99, NNZs: 1820, Bias: -1.606699, T: 77866, Avg. loss: 0.004137
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 8.27, NNZs: 2101, Bias: -1.554187, T: 116799, Avg. loss: 0.003571
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 7.99, NNZs: 2284, Bias: -1.516561, T: 155732, Avg. loss: 0.003186
Total training time: 0.04 seconds.
-- Epoch 5
Norm: 7.72, NNZs: 2420, Bias: -1.487785, T: 194665, Avg. loss: 0.002871
Total training time: 0.05 seconds.
-- Epoch 6
Norm: 7.48, NNZs: 2529, Bias: -1.470694, T: 233598, Avg. loss: 0.002703
Total training time: 0.06 seconds.
-- Epoch 7
Norm: 7.45, NNZs: 2658, Bias: -1.449424, T: 272531, Avg. loss: 0.002744
Total training time: 0.06 seconds.
Convergence after 7 epochs took 0.06 seconds
-- Epoch 1
Norm: 12.68, NNZs: 1719, Bias: -1.926332, T: 38933, Avg. loss: 0.015584
Total training time: 0.01 seconds.
-- Epoch 2
N

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


Norm: 5.45, NNZs: 938, Bias: -1.609642, T: 77866, Avg. loss: 0.001099
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 4.82, NNZs: 1093, Bias: -1.613845, T: 116799, Avg. loss: 0.000792
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 4.43, NNZs: 1231, Bias: -1.619711, T: 155732, Avg. loss: 0.000568
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 4.31, NNZs: 1342, Bias: -1.622667, T: 194665, Avg. loss: 0.000551
Total training time: 0.04 seconds.
Convergence after 5 epochs took 0.04 seconds
-- Epoch 1
Norm: 11.43, NNZs: 1464, Bias: -1.927247, T: 38933, Avg. loss: 0.011781
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 8.71, NNZs: 1907, Bias: -1.943185, T: 77866, Avg. loss: 0.002966
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 7.70, NNZs: 2218, Bias: -1.946620, T: 116799, Avg. loss: 0.001934
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 7.22, NNZs: 2462, Bias: -1.951470, T: 155732, Avg. loss: 0.001675
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 6.98, NN

Norm: 8.18, NNZs: 752, Bias: -1.256736, T: 38933, Avg. loss: 0.006130
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 6.67, NNZs: 1028, Bias: -1.220751, T: 77866, Avg. loss: 0.002496
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 6.18, NNZs: 1271, Bias: -1.203658, T: 116799, Avg. loss: 0.002030
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 5.97, NNZs: 1474, Bias: -1.195374, T: 155732, Avg. loss: 0.001888
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 5.79, NNZs: 1624, Bias: -1.188523, T: 194665, Avg. loss: 0.001767
Total training time: 0.03 seconds.
-- Epoch 6
Norm: 5.75, NNZs: 1791, Bias: -1.180829, T: 233598, Avg. loss: 0.001724
Total training time: 0.04 seconds.
Convergence after 6 epochs took 0.04 seconds
-- Epoch 1
Norm: 10.59, NNZs: 1330, Bias: -1.813268, T: 38933, Avg. loss: 0.010910
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 8.10, NNZs: 1802, Bias: -1.827345, T: 77866, Avg. loss: 0.003657
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 7.24, NNZ

Norm: 5.09, NNZs: 1639, Bias: -1.190262, T: 194665, Avg. loss: 0.001456
Total training time: 0.04 seconds.
Convergence after 5 epochs took 0.04 seconds
-- Epoch 1
Norm: 8.78, NNZs: 1017, Bias: -1.472587, T: 38933, Avg. loss: 0.008687
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 6.80, NNZs: 1351, Bias: -1.470320, T: 77866, Avg. loss: 0.003620
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 6.26, NNZs: 1583, Bias: -1.462039, T: 116799, Avg. loss: 0.003028
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 5.88, NNZs: 1732, Bias: -1.460814, T: 155732, Avg. loss: 0.002665
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 5.74, NNZs: 1818, Bias: -1.458327, T: 194665, Avg. loss: 0.002544
Total training time: 0.04 seconds.
-- Epoch 6
Norm: 5.69, NNZs: 1880, Bias: -1.451853, T: 233598, Avg. loss: 0.002429
Total training time: 0.05 seconds.
-- Epoch 7
Norm: 5.63, NNZs: 1951, Bias: -1.450051, T: 272531, Avg. loss: 0.002358
Total training time: 0.05 seconds.
Convergence after 7 epoc

...

Norm: 11.16, NNZs: 4582, Bias: -2.083096, T: 272531, Avg. loss: 0.004270
Total training time: 0.03 seconds.
-- Epoch 8
Norm: 11.12, NNZs: 4666, Bias: -2.065177, T: 311464, Avg. loss: 0.004188
Total training time: 0.03 seconds.
Convergence after 8 epochs took 0.03 seconds
-- Epoch 1
Norm: 10.17, NNZs: 1132, Bias: -1.849526, T: 38933, Avg. loss: 0.009379
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 7.71, NNZs: 1454, Bias: -1.860770, T: 77866, Avg. loss: 0.002357
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 6.87, NNZs: 1627, Bias: -1.857387, T: 116799, Avg. loss: 0.001776
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 6.41, NNZs: 1745, Bias: -1.853180, T: 155732, Avg. loss: 0.001417
Total training time: 0.02 seconds.
-- Epoch 5
Norm: 6.14, NNZs: 1856, Bias: -1.849335, T: 194665, Avg. loss: 0.001290
Total training time: 0.03 seconds.
-- Epoch 6
Norm: 6.03, NNZs: 1920, Bias: -1.847930, T: 233598, Avg. loss: 0.001221
Total training time: 0.04 seconds.
Convergence after 6 e

Norm: 5.63, NNZs: 953, Bias: -1.642765, T: 77866, Avg. loss: 0.002546
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 5.13, NNZs: 1139, Bias: -1.639831, T: 116799, Avg. loss: 0.002037
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 4.96, NNZs: 1269, Bias: -1.628550, T: 155732, Avg. loss: 0.001853
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 4.91, NNZs: 1394, Bias: -1.620115, T: 194665, Avg. loss: 0.001658
Total training time: 0.03 seconds.
-- Epoch 6
Norm: 4.80, NNZs: 1484, Bias: -1.616252, T: 233598, Avg. loss: 0.001589
Total training time: 0.04 seconds.
Convergence after 6 epochs took 0.04 seconds
-- Epoch 1
Norm: 6.06, NNZs: 510, Bias: -1.286436, T: 38933, Avg. loss: 0.003368
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 4.50, NNZs: 714, Bias: -1.299681, T: 77866, Avg. loss: 0.001378
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 4.11, NNZs: 868, Bias: -1.309815, T: 116799, Avg. loss: 0.001112
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 3.85, NNZs: 

Norm: 8.16, NNZs: 2762, Bias: -1.461120, T: 194665, Avg. loss: 0.004842
Total training time: 0.02 seconds.
-- Epoch 6
Norm: 8.11, NNZs: 2919, Bias: -1.442668, T: 233598, Avg. loss: 0.004546
Total training time: 0.03 seconds.
-- Epoch 7
Norm: 8.02, NNZs: 3014, Bias: -1.432138, T: 272531, Avg. loss: 0.004404
Total training time: 0.03 seconds.
-- Epoch 8
Norm: 8.02, NNZs: 3086, Bias: -1.417849, T: 311464, Avg. loss: 0.004310
Total training time: 0.04 seconds.
Convergence after 8 epochs took 0.04 seconds
-- Epoch 1
Norm: 21.48, NNZs: 4194, Bias: -3.808680, T: 38933, Avg. loss: 0.050803
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 16.12, NNZs: 5195, Bias: -3.758511, T: 77866, Avg. loss: 0.006771
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 14.58, NNZs: 5735, Bias: -3.719748, T: 116799, Avg. loss: 0.004659
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 14.02, NNZs: 6159, Bias: -3.693983, T: 155732, Avg. loss: 0.003966
Total training time: 0.02 seconds.
-- Epoch 5
Norm: 13.

Norm: 9.05, NNZs: 3962, Bias: -1.541163, T: 311464, Avg. loss: 0.003844
Total training time: 0.04 seconds.
-- Epoch 9
Norm: 8.96, NNZs: 4052, Bias: -1.528947, T: 350397, Avg. loss: 0.003760
Total training time: 0.04 seconds.
Convergence after 9 epochs took 0.04 seconds
-- Epoch 1
Norm: 20.05, NNZs: 3834, Bias: -2.510608, T: 38933, Avg. loss: 0.039635
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 16.06, NNZs: 4917, Bias: -2.392795, T: 77866, Avg. loss: 0.010905
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 14.72, NNZs: 5369, Bias: -2.308211, T: 116799, Avg. loss: 0.007877
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 14.15, NNZs: 5699, Bias: -2.251113, T: 155732, Avg. loss: 0.006971
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 13.69, NNZs: 5940, Bias: -2.219605, T: 194665, Avg. loss: 0.006456
Total training time: 0.03 seconds.
-- Epoch 6
Norm: 13.50, NNZs: 6104, Bias: -2.182621, T: 233598, Avg. loss: 0.005886
Total training time: 0.04 seconds.
-- Epoch 7
Norm: 1

[Parallel(n_jobs=1)]: Done 141 out of 141 | elapsed:    6.1s finished


In [48]:
from sklearn.metrics import classification_report

print(classification_report(y_train,train_preds))

             precision    recall  f1-score   support

        001       0.00      0.00      0.00       336
        011       0.00      0.00      0.00       227
        012       0.00      0.00      0.00       234
        013       0.00      0.00      0.00        62
        021       0.00      0.00      0.00       162
        031       0.00      0.00      0.00       105
        041       0.00      0.00      0.00       327
        042       0.00      0.00      0.00       215
        043       0.00      0.00      0.00        71
        051       0.00      0.00      0.00       230
        060       0.00      0.00      0.00        31
        062       0.00      0.00      0.00       113
        063       0.00      0.00      0.00        79
        065       0.00      0.00      0.00        78
        071       0.00      0.00      0.00       131
        073       0.00      0.00      0.00       109
        081       0.00      0.00      0.00        37
        082       0.00      0.00      0.00   

/home/mgooch/anaconda3/envs/insight_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [49]:
print(classification_report(y_val,val_preds))

             precision    recall  f1-score   support

        001       0.00      0.00      0.00        84
        011       0.00      0.00      0.00        57
        012       0.00      0.00      0.00        58
        013       0.00      0.00      0.00        15
        021       0.00      0.00      0.00        41
        031       0.00      0.00      0.00        26
        041       0.00      0.00      0.00        82
        042       0.00      0.00      0.00        54
        043       0.00      0.00      0.00        18
        051       0.00      0.00      0.00        57
        060       0.00      0.00      0.00         8
        062       0.00      0.00      0.00        28
        063       0.00      0.00      0.00        20
        065       0.00      0.00      0.00        20
        071       0.00      0.00      0.00        33
        073       0.00      0.00      0.00        27
        081       0.00      0.00      0.00        10
        082       0.00      0.00      0.00   

/home/mgooch/anaconda3/envs/insight_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
